In [310]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [216]:
def get_stats(stock_list):
    result = pd.DataFrame(columns=['Ticker', 'Company Name', 'P/E', 'Forward P/E',
                                   'Market Cap', 'Division'])
    for stock in stock_list:
        params = {'t': stock}
        r = requests.get('https://finviz.com/quote.ashx', params = params)
        soup = BeautifulSoup(r.text, 'lxml')
        table = soup.find('table', {'class': 'snapshot-table2'})
        table2 = soup.find('table', {'class': 'fullview-title'})
        # statistics
        Name = table2.find_all('tr')[1].find_all('a')[0].text
        P_E = float(table.find_all('td')[3].text)
        Forward_P_E = float(table.find_all('td')[15].text)
        MarCap = float(re.findall('\d+[.]\d+', table.find_all('td')[13].text)[0])
        MarCap_d = re.findall('[^\d+]', table.find_all('td')[13].text)[1]
        # final table
        result = result.append({'Ticker': stock, 'Company Name': Name,'P/E': P_E, 
                                'Forward P/E': Forward_P_E,
                                'Market Cap': MarCap, 'Division': MarCap_d}, 
                      ignore_index = True)
    return result

In [307]:
stock_list = ['AAPL', 'FB']
stats = get_stats(stock_list)

In [308]:
stats

,Ticker,Company Name,P/E,Forward P/E,Market Cap,Division
0,AAPL,Apple Inc.,17.09,13.33,909.86,B
1,FB,"Facebook, Inc.",26.70,18.73,484.44,B


In [304]:
# Recommendations lookup
def recommendations(stock):
    params = {'t': stock}
    r = requests.get('https://finviz.com/quote.ashx', params = params)
    soup = BeautifulSoup(r.text, 'lxml')
    table3 = soup.find('table', {'class': 'fullview-ratings-outer'})
    # Upgrade Recommendations
    up = pd.DataFrame()
    upgrade = table3.find_all('tr', {'class': 'body-table-rating-upgrade'})
    for i in range(1,len(upgrade),2):
        # Source data
        Date = upgrade[i].find_all('td')[0].text
        Recoomendation_cur = upgrade[i].find_all('td')[1].text
        Broker = upgrade[i].find_all('td')[2].text
        Recommendation_change = upgrade[i].find_all('td')[3].text
        Target_price_change = upgrade[i].find_all('td')[4].text
        # import data to dataframe
        up = up.append({'Date':Date, 'Current': Recoomendation_cur,
                         'Broker': Broker, 'Change': Recommendation_change,
                         'Target price change': Target_price_change}, ignore_index = True)
    
    # Neutral Recommendations
    neut = pd.DataFrame()
    neutral = table3.find_all('tr', {'class': 'body-table-rating-neutral'})
    for i in range(1,len(neutral),2):
        # Source Data
        Date = neutral[i].find_all('td')[0].text
        Recoomendation_cur = neutral[i].find_all('td')[1].text
        Broker = neutral[i].find_all('td')[2].text
        Recommendation_change = neutral[i].find_all('td')[3].text
        Target_price_change = neutral[i].find_all('td')[4].text
        # import data to dataframe
        neut = neut.append({'Date':Date, 'Current': Recoomendation_cur,
                         'Broker': Broker, 'Change': Recommendation_change,
                         'Target price change': Target_price_change}, ignore_index = True)
    
    # Downgrade Recommendations
    d = pd.DataFrame()
    down = table3.find_all('tr', {'class': 'body-table-rating-downgrade'})
    for i in range(1,len(down),2):
        # Source Data
        Date = down[i].find_all('td')[0].text
        Recoomendation_cur = down[i].find_all('td')[1].text
        Broker = down[i].find_all('td')[2].text
        Recommendation_change = down[i].find_all('td')[3].text
        Target_price_change = down[i].find_all('td')[4].text
        # import data to dataframe
        d = d.append({'Date':Date, 'Current': Recoomendation_cur,
                         'Broker': Broker, 'Change': Recommendation_change,
                         'Target price change': Target_price_change}, ignore_index = True)   
    result = pd.concat([up, neut, d], ignore_index = True)
    return result

In [305]:
a = recommendations('AAPL')

In [306]:
a

,Broker,Change,Current,Date,Target price change
0,Rosenblatt,Neutral → Buy,Upgrade,Nov-03-17,$180
1,Pacific Crest,Sector Weight → Overweight,Upgrade,Oct-16-17,$187
2,Monness Crespi & Hardt,Buy,Initiated,Apr-11-18,$235
3,Barclays,Equal Weight,Reiterated,Mar-16-18,$170 → $168
4,Raymond James,Mkt Perform,Resumed,Feb-16-18,
5,Cascend Securities,Buy,Reiterated,Jan-29-18,$200 → $220
6,Canaccord Genuity,Buy,Reiterated,Jan-24-18,$195 → $200
7,Mizuho,Neutral,Reiterated,Jan-19-18,$160 → $175
8,Maxim Group,Buy,Reiterated,Jan-12-18,$193 → $204
9,Wells Fargo,Market Perform,Resumed,Nov-15-17,$195
